In [4]:
!pip install datasets

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 95.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 84.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.9.0
    Uninstalling fsspec-2024.9.0:
      Successfully uninstalled fsspec-2024.9.0


In [2]:
from datasets import load_dataset

ds = load_dataset("HuggingFaceH4/ultrachat_200k")
print(ds.column_names)

/data/rdharmani/anaconda3/envs/persona_project/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{'train_sft': ['prompt', 'prompt_id', 'messages'], 'test_sft': ['prompt', 'prompt_id', 'messages'], 'train_gen': ['prompt', 'prompt_id', 'messages'], 'test_gen': ['prompt', 'prompt_id', 'messages']}


In [3]:
from sentence_transformers import SentenceTransformer# Define the model and tokenizer

model = SentenceTransformer('BAAI/bge-large-zh-v1.5')
embeddings_prompt_ultrachat = []
for d in ds['train_sft']['prompt'][:50]:
    embeddings_prompt_ultrachat.append(model.encode(d, normalize_embeddings=True))

In [22]:
len(embeddings_prompt_ultrachat)

50

In [4]:
import pandas as pd

implic_rea_enjoy = pd.read_json('implic_rea_enjoy.jsonl', lines=True)
print(implic_rea_enjoy['inquiry'])

0          What other activities can this person pursue?
1           What other activities can this person enjoy?
2          What other activities can this person pursue?
3          What other activities can this person pursue?
4          What low-impact activity would you recommend?
                             ...                        
890    What activity can replace their wine drinking ...
891                What activity can they enjoy instead?
892    What activity can this person enjoy instead of...
893       What alternative activity would you recommend?
894    What activity would you recommend instead of d...
Name: inquiry, Length: 895, dtype: object


In [21]:
embeddings_prompt_hobbies = []
for d in implic_rea_enjoy['inquiry']:
    embeddings_prompt_hobbies.append(model.encode(d, normalize_embeddings=True))


895

In [23]:
embeddings_prompt_hobbies[:5]

[array([ 0.03772923, -0.01813181, -0.01577174, ..., -0.03295016,
         0.01745998,  0.00290534], dtype=float32),
 array([ 0.05496497, -0.033416  , -0.00135136, ..., -0.0307684 ,
        -0.00839267,  0.0242253 ], dtype=float32),
 array([ 0.03772923, -0.01813181, -0.01577174, ..., -0.03295016,
         0.01745998,  0.00290534], dtype=float32),
 array([ 0.03772923, -0.01813181, -0.01577174, ..., -0.03295016,
         0.01745998,  0.00290534], dtype=float32),
 array([ 0.03043927,  0.00271214, -0.01939347, ..., -0.01488458,
         0.00950599,  0.0062004 ], dtype=float32)]

In [18]:
import numpy as np

embeddings_prompt_hobbies_np = np.array(embeddings_prompt_hobbies)
embeddings_prompt_ultrachat_np = np.array(embeddings_prompt_ultrachat)

In [33]:
similarities_prompt = []
for prompt_ultrachat in embeddings_prompt_hobbies_np:
    similarities_prompt.append(embeddings_prompt_hobbies_np @ prompt_ultrachat)

In [42]:
similarities_prompt_np = np.array(similarities_prompt)
most_similar_prompt_index = np.argmax(similarities_prompt_np, axis=1)

In [46]:
most_similar_prompt_index[:5]

array([0, 1, 0, 0, 4])

In [48]:
implic_rea_enjoy['inquiry'][most_similar_prompt_index[4]]


'What low-impact activity would you recommend?'